# Getting started with langchain - question answering

This notebook walks through how to use LangChain for question answering over a given document. 

### Sources
- PyPi: https://pypi.org/project/langchain/
- GitHub: https://github.com/hwchase17/langchain
- Documents: https://python.langchain.com/en/latest/use_cases/question_answering.html

### Contents
0. Install packages
1. Prepare data
2. First langchain call

## 0. Install packages

In [ ]:
!pip install langchain


## 1. Prepare Data
First we prepare the data. For this example we do similarity search over a vector database, but these documents could be fetched in any manner (the point of this notebook to highlight what to do AFTER you fetch the documents).

In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.indexes.vectorstore import VectorstoreIndexCreator
import config

In [6]:
#store your openai api key as an environment variable - as required by OpenAI
import os
#os.environ['OPENAI_API_KEY'] = 'YOUR API KEY HERE' #but don't keep it in your source code :-)

In [7]:
with open("state_of_the_union.txt") as f:
    state_of_the_union = f.read()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(state_of_the_union)

embeddings = OpenAIEmbeddings()

In [8]:
docsearch = Chroma.from_texts(texts, embeddings, metadatas=[{"source": str(i)} for i in range(len(texts))]).as_retriever()

Using embedded DuckDB without persistence: data will be transient


In [9]:
query = "What did the president say about Justice Breyer"
docs = docsearch.get_relevant_documents(query)

In [12]:
docs[0]

Document(page_content='Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.', metadata={'source': '31'})

In [10]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

## 2. First langchain call
If you just want to get started as quickly as possible, this is the recommended way to do it:

In [15]:
chain = load_qa_chain(OpenAI(temperature=0), chain_type="stuff")
query = "What did the president say about Justice Breyer"
chain.run(input_documents=docs, question=query)

' The president said that Justice Breyer is an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court, and thanked him for his service.'